In [1]:
# Audio Play

import pyaudio
import wave
import time
import sys

def play_audio(filename):    
    WAVE_FILENAME = filename
    if len(sys.argv) < 2:
        print("Plays a wave file.\n\nUsage: %s filename.wav" % WAVE_FILENAME)
        sys.exit(-1)

    wf = wave.open(WAVE_FILENAME, 'rb')

    p = pyaudio.PyAudio()

    def callback(in_data, frame_count, time_info, status):
        data = wf.readframes(frame_count)
        return (data, pyaudio.paContinue)

    stream = p.open(format=p.get_format_from_width(wf.getsampwidth()),
                    channels=wf.getnchannels(),
                    rate=wf.getframerate(),
                    output=True,
                    stream_callback=callback)

    stream.start_stream()

    while stream.is_active():
        time.sleep(0.1)

    stream.stop_stream()
    stream.close()
    wf.close()

    p.terminate()

In [2]:
import numpy as np
import cv2
face_cascade = cv2.CascadeClassifier('files/haarcascade/haarcascade_frontalface_default.xml')
profile_cascade = cv2.CascadeClassifier('files/haarcascade/haarcascade_profileface.xml')
eye_cascade = cv2.CascadeClassifier('files/haarcascade/haarcascade_eye.xml')
glass_cascade = cv2.CascadeClassifier('files/haarcascade/haarcascade_eye_tree_eyeglasses.xml')
smile_cascade = cv2.CascadeClassifier('files/haarcascade/haarcascade_smile.xml')

def cascade_pipe(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(gray,
                                          scaleFactor=1.1,
                                          minNeighbors=5,
                                          minSize=(30, 30))
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray)
        glasses = glass_cascade.detectMultiScale(roi_gray)
        smiles = smile_cascade.detectMultiScale(roi_gray, minNeighbors=65)
        for (ex,ey,ew,eh) in eyes:
            cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
        for (gx,gy,gw,gh) in glasses:
            cv2.rectangle(roi_color,(gx,gy),(gx+gw,gy+gh),(0,255,255),2)
        for (sx,sy,sw,sh) in smiles:
            cv2.rectangle(roi_color,(sx,sy),(sx+sw,sy+sh),(0,0,255),2)

    profile_faces = profile_cascade.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in profile_faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,255,0),2)
    return img

In [8]:
## Webcam test
cap = cv2.VideoCapture(0)

while True:
    ret, img = cap.read()
    img_c = cascade_pipe(img)
    cv2.imshow('img',img)
    k = cv2.waitKey(30) & 0xff
    if k==97:
        play_audio("mujer.wav")
    if k==27:
        break
        
cap.release()
cv2.destroyAllWindows()

In [4]:
play_audio("mujer.wav")